# Proyecto: Detección de pérdida de clientes

Imaginemos que trabajamos para una compañía de telecomunicaciones, la cual ofrece servicios de telefonía e internet. La compañía atraviesa por un problema: algunos de los clientes la están abandonando (*churn*). Mencionan que ya no requieren de los servicios y que se irán con un proveedor diferente. A la empresa le gustaría evitar que esto suceda, así que vamos a desarrollar un sistema para identificar a estos clientes y ofrecerles un incentivo para que permanezcan con la compañía. 

<center>
    <figure>
    <img src = "https://www.salesforce.com/mx/blog/wp-content/uploads/sites/11/2023/09/churn-rate.jpg" alt="Proyecto de un sistema de clasificación de clientes potenciales a abandonar los servicios de una empresa." style="width:500px;height:350px;">
    <figcaption> ¿Cómo identificar a clientes que potencialmente podrían abandonar los servicios de  una compañía y qué hacer para evitarlo? </figcaption>
    </figure>
</center>

Tenemos disponible un conjunto de datos donde se ha registrado información sobre los clientes: el tipo de servicio que contrataron, cuánto pagan por él, y qué tanto tiempo han permanecido como clientes. También, disponemos de la información de quiénes han cancelado sus contratos y dejado de utilizar los servicios. 

## Estrategia

 1. Primero, descargamos el conjunto de datos y hacemos una preparación inicial: renombraremos columnas y cambiaremos los valores de las filas para que sean consistentes durante todo el proyecto.

 2. Después, dividiremos los datos en los conjuntos de entrenamiento, validación y prueba.

 3. Como parte del análisis de datos inicial, debemos ver la importancia de las características para identificar cuáles son importantes en nuestros datos.

 4. Transformamos variables categóricas a numéricas para utilizarlas en el modelo.

 5. Finalmente, entrenaremos un modelo de regresión logística.


## Obteniendo los datos

Para este proyecto, utilizamos el conjunto de datos [Telco Customer Churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn) de [Kaggle](https://www.kaggle.com/). De acuerdo con la deescripción, el dataset contiene la siguiente información:

 - Servicios de los clientes: teléfono, múltiples líneas, internet, soporte técnico y servicios adicionales como seguridad online, backup, protección de dispositivos y TV streaming.
 - Información de la cuenta: qué tanto tiempo han sido clientes, tipo de contrato y tipo de método de pago.
 - Cargos: el monto de pago del cliente del mes anterior y en total.
 - Información demográfica: género, edad y si dependen de ellos o tienen pareja.
 - Churn: si/no, el cliente ha dejado la compañía dentro del mes anterior.

El archivo se incluye en la carpeta `data` del repositorio. 

## Preparación inicial de los datos

Comencemos importando las bibliotecas necesarias para desarrollar el proyecto.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Cargamos el dataset.

In [ ]:
data = pd.read_csv('data/customer-churn.csv')

data.head(5)

Veamos la cantidad de datos y columnas que tenemos en el dataframe.

In [ ]:
data.shape

Tenemos 7043 registros y 21 columnas. Veamos ahora qué columas tenemos y qué significan.

In [ ]:
data.columns

Echemos un vistazo a lo que representa cada una y sus posibles valores.

 - `customerID`: el número de ID del cliente.
 - `gender`: masculino/femenino.
 - `SeniorCitizen`: si el cliente es o no un adulto mayor (0/1).
 - `Partner`: si el cliente vive con su pareja o no (yes/no).
 - `Dependents`: si el cliente tiene personas que dependan económicamente de él (yes/no).
 - `tenure`: el número de meses que han pasado desde el inicio del contrato.
 - `PhoneService`: si el cliente tiene o no servicio telefónico (yes/no)
 - `MultipleLines`: si el cliente tiene o no múltiples líneas telefónicas (yes/no/no phone service).
 - `InternetService`: el tipo de servicio de internet (no/fiber/optic).
 - `OnlineSecurity`: si el servicio de seguridad online está activo (yes/no/no internet).
 - `OnlineBackup`: si el servicio de backup está activo (yes/no/no internet).
 - `DeviceProtection`: si el servicio de protección de dispositivos está activo (yes/no/no internet).
 - `TechSupport`: si el cliente cuenta con soporte técnico (yes/no/no internet).
 - `StreamingTV`: si el servicio de streaming TV está activo (yes/no/no internet).
 - `StreamingMovies`: si el servicio de streaming de pelícuas está activo (yes/no/no internet).
 - `Contract`: tipo de contrato (monthly/yearly/two years).
 - `PaperlessBilling`: si la factura es electrónica (yes/no).
 - `PaymentMethod`: método de pago (electronic check, mailed check, bank transfer, credit card).
 - `MonthlyCharges`: la cantidad cobrada mensualmente (numérico).
 - `TotalCharges`: la cantidad cobrada en total (numérico).
 - `Churn`: si el cliente ha cancelado el contrato (yes/no).

 La característica que nos importa más es la última, `Churn`, ya que es la variable objetivo para nuestro modelo, es decir, lo que nos interesa predecir. Notemos que toma dos posibles valores, `yes` si el cliente se fue o `no` si no lo hizo.

 
Ahora, generalmente, cuando Pandas lee un archivo CSV, automáticamente intenta determinar el tipo adecuado de variable para cada columna. Pero en ocasiones no lo hace, así que debemos verificar si el tipo de datos es correcto.


In [ ]:
data.dtypes

Tenemos distintos tipos de variables: `object`, `int64` y `float64`. Las variables tipo `object` suelen referirse a valores string. Por otro lado, los datos tipo `int64` y `float64`, se refieren a variables numéricas, enteros y flotantes de 64 bits.

Es por ello, que tras una inspección visual, notamos que la variable `TotalCharges` es del tipo `object`, pero al ser un monto de dinero, debería ser numérica. En ocasiones esta confusión ocurre porque Pandas infiere que el tipo de dato es `object` debido a que la columna contiene espacios en blanco para representar un valor faltante. 

Podemos corregir esta situación, al convertirla a una columna numérica, con la función de pandas `to_numeric()`. Algo a considerar es que por default, esta función señala una excepción cuando ve datos no numéricos (como los espacios en blanco). Para evitarlo, especificamos el con el argumento `errors = 'coerce'`. Pandas reemplazará los valores no numéricos con un valor `NaN`. 

In [ ]:
total_charges = pd.to_numeric(data.TotalCharges, errors = 'coerce')

total_charges

Para confirmar que en efecto los datos contenían caracteres no numéricos, podemos utilizar la función `is_null()`de `total_charges` y usarla para referiros a todas las columnas en las que Pandas no pudo transformar la string original.

In [ ]:
data[total_charges.isnull()][['customerID', 'TotalCharges']]

Estos son los registros de la dataframe en los que la columna `TotalCharges` tiene espacios en blanco como valores. Ahora, ¿qué hacemos con ellos? Por simplicidad, haremos que los valores sean iguales a 0.

In [ ]:
data.TotalCharges = pd.to_numeric(data.TotalCharges, errors = 'coerce')

data.TotalCharges = data.TotalCharges.fillna(0)

Adicionalmente, notamos que los nombres de la columna no siguen una convención en específico. Algunas inician con minúsculas, otras con mayúsculas, etc. 
Para mantener uniformidad, vamos a hacer que todos los nombres de las columnas estén en minúsculas, y reemplazar cualquier espacio con guiones bajos.

In [ ]:
data.columns = data.columns.str.lower().str.replace(' ', '_')

string_cols = list(data.dtypes[data.dtypes == 'object'].index)

for col in string_cols:

    data[col] = data[col].str.lower().str.replace(' ', '_')

In [ ]:
data.head()

Ahora consideremos la variable objetivo, `churn`.

In [ ]:
data.churn

Como mencionamos anteriormente, tiene dos posibles valores, `yes` o `no`. Notemos que se trata de una variable categórica. Para clasificación binaria, todos los modelos generalmente esperan recibir un número, `0` para no y `1` para yes.

Es necesario hacer la conversión numérica.

In [ ]:
data.churn = (data.churn == 'yes').astype(int)

data.churn

Hemos hecho un poco de pre-procesamiento de datos, pero hagamos un par de pruebas con lo que tenemos. En esta ocasión, utilizaremos la librería Scikit-Learn para la división de los conjuntos y para el entrenamiento e implementación del modelo. Primero, importamos el módulo `model_selection` que puede realizar las tareas de dividir los datos. La función que utilizaremos de este módulo se llama `train_test_split`.

In [ ]:
from sklearn.model_selection import train_test_split

Una vez que importamos la función, podemos utilizarla. Esta función toma como entrada la `dataframe` que contiene nuestros datos y crea dos nuevos conjuntos: el conjunto de entrenamiento y el conjunto de prueba. La función lo hace mezclando las entradas en el dataset original para posteriormente dividirlo de tal forma que el data test contenga un porcentaje de los datos y el data train el porcentaje restante. 

Probemos con un data train que contenga el 80% de los datos y que el data test el 20% restante.

In [ ]:
data_train_full, data_test = train_test_split(data, test_size = 0.2, random_state=42)

<center>
    <figure>
    <img src = "img/train_test_flow.png" style="width:650px;height:375px;">
    <figcaption> El dataset original se mezcla y después se divide de tal forma que el 80% de los datos va al conjunto de entrenamiento y el 20% restante al conjunto de prueba. </figcaption>
    </figure>
</center>

El tercer parámetro de la función `train_test_split()`, `random_state` se utiliza para garantizar que cada vez que corramos el código, la dataframe se divida de la misma forma siempre. 

Veamos cómo se ve el conjunto de entrenamiento.

In [ ]:
data_train_full.head()

Hasta el momento vamos bien, pero sabemos que debemos dividir a los datos en 3 conjuntos: entrenamiento, validación y prueba. La función `train_test_split()` sólo los divide en entrenamiento y prueba. Podemos dividir el conjunto de entrenamiento obtenido usando la misma función para generar el conjunto de validación.

In [ ]:
data_train, data_val = train_test_split(data_train_full, test_size=0.33, random_state=69)

Ahora debemos separar a la columna que contiene a la variable objetivo y almacenarla de forma separada a los dataframe de entrenamiento y validación. 

In [ ]:
y_train = data_train.churn.values

y_val = data_val.churn.values

In [ ]:
del data_train['churn']
del data_val['churn']

Con los dataframe preparados, podemos utilizar el conjunto de entrenamiento para realizar un análisis exploratorio de datos.

## Análisis exploratorio de datos

Primero, veamos si tenemos valores nulos en el dataset.

In [ ]:
data_train.isnull().sum()

Afortunadamente, ninguna columna contiene datos nulos, así que no hay nada más que hacer.

Por otro lado, podemos revisar la distribución de los valores en la variable objetivo. Primero, usamos la función `value_counts()`.

In [ ]:
data_train_full.churn.value_counts()

De los 5634 registros en el primer conjunto de entrenamiento, 4138 no se han ido mientras que 1496 sí, eso significa que alrededor del 27% de los clientes dejaron de utilizar los servicios. Esto representa la probabilidad de que un cliente abandone la compañía, por lo que este número se conoce como *tasa de cancelación de clientes* o *churn rate*.

Existe otra forma de calcular el *churn rate*, y es a través del promedio. 

In [ ]:
global_avg = data_train_full.churn.mean()

print(f"Churn rate: {np.round(global_avg,3)}")

Bien, regresando a la cantidad de clientes que abandonaron, es notorio que una categoría tiene muchos más elementos que la otra. Esto es un ejemplo de lo que se conoce como *datasets desbalanceados* (*imbalanced*), por lo que decimos que una clase domina a la otra. El opuesto de esta situación es un dataset *balanceado*, donde las clases positivas y negativas se distribuyen equitativamente en todas las observaciones.

Más adelante veremos qué se puede hacer en casos donde el desbalanceo de categorías juega un papel importante en los resultados del modelo.

Por el momento continuaremos con el análisis. Es importante considerar que tanto las variables categóricas como numéricas del conjunto de datos son importantes, pero también son distintas y necesitan un tratamiento diferente. Por lo tanto, vamos a manejarlas de manera separada. Crearemos dos listas: `numerical` y `categorical`. La primera contendrá todas las variables categóricas y la segunda las numéricas, excluyendo la característica `customerid`.

Tambien, hay que recordar que a pesar de que `seniorcitizen` contiene datos numéricos, la consideramos una variable categórica. 

Para hacerlo, en vez de escribir los nombres uno por uno, utilizamos el método de *list comprehension*, el cual nos proporciona una forma concisa y eficiente para crear listas. Utiliza una expresión para generar los elementos de la lista, lo que nos permite que se apliquen operaciones y filtros a los elementos de una secuencia o iterable. Las *list comprehension* muchas veces pueden reemplazar a los ciclos `for` para generar una lista.

La sintaxis básica de una list comprehension es la siguiente:

    list = [expression for item in iterable if condition]

- `expression`: la expresión que produce los elementos de la nueva lista, puede ser cualquier tipo de operación válida en Python, incluyendo la llamada de funciones. Además, la expresión puede depender de la variable item.
- `item`: la variable que toma cada valor del `iterable` en cada iteración.
- `iterable`: secuencia o colección sobre la que se itera, puede ser una lista, un rango, una string, etc.
- `condition` (opcional): una condición que filtra los elementos del `iterable`, por lo que sólo se incluirán en la nueva lista los elementos para los que la condición sea verdadera.

Ejemplo: Crear una lista de los cuadrados de los números del 0 al 9:

In [ ]:
squares = [x**2 for x in range(10)]

print(squares)

El ciclo `for` correspondiente sería:

In [ ]:
sq = []

for x in range(10):

    sq.append(x**2)

print(sq)

Ejemplo: Filtrar números impares del 0 al 9 y calcular sus cubos:

In [ ]:
cubic_odds = [x**3 for x in range(10) if x % 2 == 1]

print(cubic_odds)

Entonces, creamos las listas `categorical` y `numerical` con list comprehension así:

In [ ]:
categorical = []

numerical = []

categorical = [col for col in data_train_full.columns if data_train_full[col].dtype == 'object'] # Filtramos las columas cuyo dtype sea `object`

numerical = [col for col in data_train_full.columns if data_train_full[col].dtype in ['float64', 'int64'] and col != 'seniorcitizen'] # Filtramos las columnas cuyo dtype sea `float64` o `int64` excluyendo a seniorcitizen

categorical = [col for col in categorical if col != 'customerid'] + ['seniorcitizen'] # añadimos `seniorcitizen` a la lista de categóricas

numerical = [col for col in numerical if col!= 'customerid'] # excluimos la columna `customerid` de la lista numérica

print('Columnas categóricas: ', categorical)
print('Columnas numéricas: ', numerical)

Ahora, podemos ver cuántos valores únicos tiene cada columna.

In [ ]:
data_train_full[categorical].nunique()

La mayoría de las columnas categóricas tienen entre 2 y 3 categorías. Ahora tenemos que hacer otra parte importante del análisis exploratorio de datos: entender cuáles características podrían ser importantes para el modelo. 

## Feature importance

El saber cómo el resto de las variables afecta a la variable objetivo, `churn`, es fundamental para entender los datos y construir un buen modelo. Este proceso se conoce como `feature importance analysis` y generalmente se hace como parte del análisis exploratorio de datos para averiguar qué variables nos serán útiles. También, nos da perspectivas adicionales acerca del conjunto de datos y nos ayudará a responder preguntas como "¿Qué hace que los clientes abandonen los servicios de la empresa?" y, "¿qué características comparten la gente que abandona?".

Comencemos con las variables categóricas. Lo primero que podemos hacer es ver el churn rate para cada variable. Sabemos que para cada variable, hay un grupo de clientes que comparten ese valor. Para cada grupo, podemos calcular el churn rate, que corresponderá al valor de ese grupo. Cuando lo tengamos, podemos compararlo con el churn rate global.

Si la diferencia entre estas tasas es pequeña, el valor no es importante al momento de predecir la tasa de abandono porque este grupo de clientes no es muy diferente del resto. Por otro lado, si la diferencia es considerable, algo dentro de ese grupo los separa del resto. Un modelo de machine learning debería ser capaz de identificar esto y usarlo para generar predicciones.

Veamos primero la variable `gender`. La cual puede tomar dos valores, `female` y `male`. Por lo tanto hay dos grupos de clientes, aquellos para los cuales se cumple `gender == female` y para los que se cumple `gender == male`. 

Calculemos entonces el churn rate para cada grupo.

In [ ]:
female_mean = data_train_full[data_train_full.gender == 'female'].churn.mean()

print(f"El churn rate para el grupo femenino es de: {np.round(female_mean, 3)}")

In [ ]:
male_mean = data_train_full[data_train_full.gender == 'male'].churn.mean()

print(f"El churn rate para el grupo masculino es de: {np.round(male_mean, 3)}")

In [ ]:
print(f"Churn rate global: {np.round(global_avg,3)}")

Resulta que el churn rate de las clientas es de alrededor del 27.7%, mientras que el de los clientes es de alrededor del 26%. Recordemos que el churn rate global es del 27%. 
Por lo tanto, la diferencia entre las tasas de ambos grupos es pequeña, lo que nos indica que conocer el género del cliente no nos ayuda a identificar si abandonarán los servicios de la compañía o no.

Intentemos con otra variable: `partner`. Esta toma los valores `yes` o `no`.

In [ ]:
partner_yes = data_train_full[data_train_full.partner == 'yes'].churn.mean()

partner_no = data_train_full[data_train_full.partner == 'no'].churn.mean()

print(f"El churn rate para partner = yes es de: {np.round(partner_yes, 3)}")
print(f"El churn rate para partner = no es de: {np.round(partner_no, 3)}")
print(f"Churn rate global: {np.round(global_avg,3)}")

En este caso, los clientes que indicaron tener pareja tienen un menor churn rate que aquellos que no la tienen. Es decir, es más probable que los clientes sin pareja abandonen la compañía.

## Risk ratio

Además de ver la diferencia entre la tasa grupal y la tasa global, resulta interesante mirar la razón entre ellas. En estadística, la razón entre las proabilidades en distintos grupos se conoce como *riesgo relativo* (*razón de riesgo* o *risk ratio*), donde el *riesgo* se refiere al riesgo de tener el efecto. En nuestro caso, el efecto es el churn, así que tenemos:

$$ \text{risk} = \frac{\text{tasa grupal}}{\text{tasa global}} $$

Entonces para `gender == female` tenemos:

In [ ]:
risk_gender_female = female_mean / global_avg

print(round(risk_gender_female,3))

El riesgo es un número entre 0 e infinito. Se interpreta como qué tan plausible es que los elementos del grupo sufran del efecto comparado con la población total. Si la diferencia entre la tasa grupal y la global es pequeña, el riesgo es cercano a 1: el grupo tiene el mismo nivel de riesgo como el resto de la población. Un grupo con un riesgo cercano a 1 no es riesgoso en absoluto.

Si el riesgo es menor a 1, el grupo tiene menores riesgos, ya que la tasa grupal es menor a la global. Por otro lado, si el valor es mayor a 1, el grupo es riesgoso: hay más churn en el grupo que en la población. Por lo que un riesgo de, por ejemplo, 2 significa que los clientes de dicho grupo son 2 veces más probables a abandonar.

Ahora calculemos los riesgos para `gender == male` y creamos un dataframe que almacene la información calculada.

In [ ]:
data_group = data_train_full.groupby(by = 'gender').churn.agg(['mean']) # Agrupamos a partir de data_train_full por género y agregamos la columna 'mean'

data_group['diff'] = data_group['mean'] - global_avg # Creamos la columna diferencia que calcula la diferencia entre la tasa grupal y la global para cada categoria

data_group['risk'] = data_group['mean'] / global_avg # Creamos la columna risk que calcula el riesgo para cada grupo

data_group

Ahora repitamos esto para cada variable categórica.

In [ ]:
from IPython.display import display # Para ver los dataframes generados en cada iteración se utiliza la función display del módulo display de IPython

for col in categorical:

    data_group = data_train_full.groupby(by = col).churn.agg(['mean'])

    data_group['diff'] = data_group['mean'] - global_avg

    data_group['risk'] = data_group['mean'] / global_avg

    display(data_group)

Entonces, a partir de los resultados tenemos lo siguiente:

 - Para `gender`: no hay diferencia considerable entre las dos categorías. Sus medias son similares y para ambos grupos, los riesgos son cercanos a 1.
 - Para `partner`: los clientes con pareja abandonan menos que los que no tienen pareja. Sus riesgos son de **0.75** y **1.22**, respectivamente.
 - Para `dependents`: los clientes sin dependientes económicos son más propensos a abandonar que los que los tienen. Sus riesgos son de **1.17** y **0.58**.
 - Para `phoneservice`: los clientes que usan el servicio telefónico no están en riesgo de abandonar: su riesgo es cercano a **1**. Los que no usan servicio telefónico son menos propensos a abandonar, su riesgo es < 1 y la diferencia con la tasa global es negativa.
 - etc.

Esta revisión 1 a 1 es tediosa. Vamos a utilizar el mismo código que genera las dataframe de diferencias y riesgos para cada variable categórica pero vamos a graficar la información de la columna `risk` para cada categoría, con la intención de visualizar qué valores de cada categoría presentan un mayor riesgo de abandonar.

In [ ]:
n_categories = len(categorical)

n_cols = 4

n_rows = 4

fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize = (20, 25)) 


for i, col in enumerate(categorical):

    ax = axes[i // n_cols, i%n_cols] if n_rows > 1 else axes[i] # Encuentra el índice del subplot actual

    data_group = data_train_full.groupby(by = col).churn.agg(['mean'])

    data_group['diff'] = data_group['mean'] - global_avg

    data_group['risk'] = data_group['mean'] / global_avg

    data_group['risk'].plot(kind = 'bar', ax = ax)

    ax.set_title(f'Riesgo de churn por {col}') # Dibujamos el gráfico de barras en el eje actual
    ax.set_xlabel(col)
    ax.set_ylabel('Riesgo relativo de Churn')


fig.tight_layout() # Ajustamos cada figura para mejor visualización

plt.show() # Muestra la figura completa con todos los subgráficos




De los gráficos podemos identificar dos variables con diferencias significativas:

 - Para `techsupport`: los clientes que no cuentan con el servicio de soporte técnico tienden a abandonar más que los que si lo tienen.
 - Para `contract`: los clientes con un contrato mes a mes son más propensos a abandonar que aquellos con contratos anuales y bianuales.
 - Para `paymentmethod`: los clientes que pagan con cheques electrónicos tienden a abandonar más que el resto de los métodos de pago.


De esta forma, al observar las diferencias y los riesgos, podemos identificar las características que resultarán más beneficiosas para detectar el churn. Por lo que esperamos que estas características sean más útiles para los modelos que construyamos.

## Mutual information

Las diferencias que exploramos son útiles para nuestro análisis e importantes para entender los datos, pero es difícil usarlas para decir cuál es la característica más importante y si `techsupport`es más importante que `contract``. 

Afortunadamente, las métricas de importancia nos pueden ayudar: podemos medir el grado de dependencia entre una variable categórica y la variable objetivo. Si dos variables son dependientes, conocer el valor de una nos da información sobre la otra. Por otro lado, si una variable es completamente independiente de la objetivo, no es útil y podemos removerla con seguridad.

Para variables categóricas, se puede emplear la métrica de *mutual information* (*información mutua*), que nos dice cuánta información aprendemos sobre una variable si aprendemos el valor de la otra. Es un concepto de teoría de la información, y aplicada en machine learning, a menudo se usa para medir la dependencia mutua entre dos variables.

Valores mayores de mutual information significan un mayor grado de dependencia, en cambio, un valor menor de mutual information indica que la variable categórica y la variable objetivo son independientes.

Mutual information ya está implementada en la función `mutual_info_score` del módulo `metrics`de Scikit-Learn:

In [ ]:
from sklearn.metrics import mutual_info_score

Crearemos una función que calcule la información mutua entre la variable categórica y la variable objetivo.

In [ ]:
def m_i(series):

    return mutual_info_score(series, data_train_full.churn)

Ahora aplicamos la función para las variables categóricas en el dataset original. Después, ordenamos los valores del resultado.

In [ ]:
data_mi = data_train_full[categorical].apply(m_i)

data_mi = data_mi.sort_values(ascending =False).to_frame(name ='MI')

data_mi

Notemos que `contract`, `onlinesecurity` y `techsupport` son las características más importantes. Lo cual es una confirmación del resultado calculado por la razón de riesgo. De la misma forma que `gender` figura como una de las características menos importantes. 

## Coeficiente de correlación

La información mutua es una manera de cuantificar el grado de dependencia entre dos variables categóricas, pero no funciona cuando una de las características es numérica. Para medir la dependencia entre una variable objetivo binaria y una numérica, podemos suponer que la variable binaria es numérica y utilizar los métodos clásicos de estadística para buscar dependencia entre las variables.

Uno de esos métodos es el *coeficiente de correlación* (también conocido como *coeficiente de Pearson*), el cual es un valor numérico entre -1 y 1:

 - Una *correlación positiva* significa que cuando una variable aumenta, la otra también. En el caso de un objetivo binario, cuando los valores de la variable sean altos, veremos más unos que ceros. Cuando los valores de la variable sean bajos, los ceros serán más frecuentes.
 - Una *correlación cero* significa que no hay relación entre las dos variables, son completamente independientes.
 - Una *correlación negativa* ocurre cuando una variable aumenta y la otra disminuye. Para el caso binario, si los valores son altos, veremos más ceros que unos en el objetivo. Cuando los valores sean bajos veremos más unos.

Podemos calcular el coeficiente de correlación en Pandas con la función `corrwith(series)`.

In [ ]:
data_train_full[numerical].corrwith(data_train_full.churn)

Los resultados indican:

 - La correlación entre `tenure` y `churn` es de **-0.35**, al ser negativo, mientras más tiempo permanezcan como clientes, tienen menor tendencia a abandonar los servicios.
 - La correlación entre `monthlycharges` y `churn`es de **0.19**, es positiva por lo que los clientes que pagan más tienden a abandonar más a menudo.
 - La correlación entre `totalcharges`y `churn` es de **-0.19**, una correlación negativa: mientras más tiempo permancen los clientes en la compañía más pagan en total, así que es menos probable que abandonen. 

 Veamoslo gráficamente.

In [ ]:
data_train_full['tenure_group'] = pd.cut(data_train_full['tenure'], bins = [0, 2, 12, np.inf], labels = ['0-2', '3-12', '>12'])

data_train_full['monthly_group'] = pd.cut(data_train_full['monthlycharges'], bins = [0, 20, 50, np.inf], labels = ['0-20', '21-50', '>50'])

tenure_churn_rate = data_train_full.groupby('tenure_gropu')['churn'].mean()*100

monthly_charges_churn_rate = data_train_full.groupby('monthly_group')['churn'].mean()*100

total_charges_churn_rate = data_train_full.groupby('totalcharges')['churn'].mean()*100

tenure_corr = data_train_full.tenure.corr(data_train_full.churn)

monthly_corr = data_train_full.monthlycharges.corr(data_train_full.churn)

total_charges_corr = data_train_full.totalcharges.corr(data_train_full.churn)

fig, axes = plt.subplots(1, 3 , figsize = (8,6))


tenure_churn_rate.plot(kind='bar', ax=axes[0], color='gray')
axes[0].set_xlabel('Tenure', fontsize = 12)
axes[0].set_ylabel('Tasa de abandono [%]')

plt.show()

